In [1]:
!pip install --upgrade pip -q
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 103

In [2]:
# Launching ElasticSearch Daemon for DocumentStores

! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)

! sleep 30

In [3]:
!cp -r /content/drive/MyDrive/news /content/

In [4]:
import pandas as pd
import seaborn as sns

dir_path = '/content/news'
files = [os.path.join(dir_path, filename) for filename in os.listdir(dir_path)]

dfs = []

for i in range(len(files)):

  temp_df = pd.read_csv(files[i])
  dfs.append(temp_df)

df = pd.concat(dfs)
df.shape

(142570, 10)

In [5]:
df['length'] = [len(value.split()) for value in list(df['content'].values)]
df = df.loc[(df['length'] > 15) & (df['length'] < 512)]

In [6]:
from haystack.document_stores import ElasticsearchDocumentStore

docs = []

for value in list(df['content'].values):

  doc = {
      'content':value
  }

  docs.append(doc)

document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

In [7]:
from haystack.nodes import QuestionGenerator
from haystack.pipelines import QuestionGenerationPipeline

question_generator = QuestionGenerator()
pipeline = QuestionGenerationPipeline(question_generator)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/195 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using sep_token, but it is not set yet.


In [8]:
document_store.describe_documents()

{'count': 60109,
 'chars_mean': 1812.5168443993412,
 'chars_max': 5952,
 'chars_min': 41,
 'chars_median': 1839.0}

In [9]:
from haystack.utils import print_questions

for idx, document in enumerate(document_store):

  print(f'\n* Generating Questions for {idx} : {document.content[:100]}\n')

  questions = pipeline.run(documents = [document])

  print_questions(questions)

  break


* Generating Questions for 0 : LONDON  —   Britain’s opposition Labour Party, already reeling after voters defied its advice and ch


Generated questions:
 - What happened to Britain's opposition Labour Party on Tuesday?
 - Who lost a motion to leave the European Union?
 - What was the result of Corbyn's vote?
 - How many votes did the measure pass?
 - What does the measure open the way for?
 - Who has refused to step down?
 - What percentage of Labour members and supporters voted for Corbyn to resign?
 - What did Corbyn say after the vote by Labour members of Parliament has no constitutional legitimacy?
 - What is the democratic party with a clear constitution?
 - Who has been hit by the mass resignation of most of his leadership team?
 - What was Britain's vote to leave the European Union?
 - What party did the majority of Parliament members oppose leaving the union?
 - Who was the prime minister when he announced he would resign?
 - When did Cameron announce his resignation?
 - Wha

In [10]:
from haystack.nodes import BM25Retriever
from haystack.pipelines import RetrieverQuestionGenerationPipeline

retriver = BM25Retriever(document_store = document_store)
retriver_generation = RetrieverQuestionGenerationPipeline(retriver,question_generator)

In [11]:
from haystack.utils import print_questions

questions = retriver_generation.run(query = 'ISTANBUL')
print_questions(questions)


Generated questions:
 - How many people were killed when a suicide bomber attacked a pedestrian street in central Istanbul?
 - How many pedestrians were killed in the attack on Istiklal Street?
 - What street is full of cafes, shops, restaurants, and foreign consulates?
 - Who is Zeynep Bilginsoy?
 - What is Zeynep Bilginsoy reporting from?
 - What time did the bomb strike?
 - Where did the suicide bomber detonate?
 - What has been cordoned off and helicopters can be heard over the city?
 - How many people were killed by a car bomb in Ankara?
 - In what month did an explosion in Istanbul kill 13 tourists?
 - How many tourists were killed in a tourist neighborhood in Istanbul?
 - What is the sixth bombing targeting civilians that has struck Turkey since June?
 - Who reports there had been warnings about violence in Istanbul leading up to the attack?
 - What did the U.S. and German consulates warn their citizens of?
 - What did Germany shut its consulate near?
 - What did the governor's

In [ ]:
from haystack.nodes import FARMReader
from haystack.pipelines import QuestionAnswerGenerationPipeline

reader = FARMReader("deepset/roberta-base-squad2")
qa_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(document_store):

    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")

    result = qag_pipeline.run(documents=[document])
    print_questions(result)

    break